### Gibbs Sampling Tutorial ###

This tutorial walks through the intuition of running Gibbs sampling for Latent Dirichlet Allocation. The method here is not the most efficient way of performing LDA, but hopefully it provides the intution for the more efficient version, known as collapsed Gibbs sampling. A nice implementation of LDA with collapsed Gibbs sampling can be found on Stephen Hansen's github at <a>https://github.com/sekhansen/text-mining-tutorial</a>. 

In [2]:
import pandas as pd
import re
from itertools import chain
import numpy as np 
from collections import Counter

data = [["rugby","football","competition","ball","games"],
        ["macro","economics","competition","games"],
        ["technology","computers","apple","AAPL","internet"],
        ["football","score","touchdown","team"],
        ["keynes","macro","friedman","policy"],
        ["stocks","AAPL","gains","analysis"],
        ["playoffs","games","season","compete","ball"],
        ["analysis","economy","economics","government"],
        ["apple","team","jobs","compete","computers"]]

### Roadmap ###

We would like to extract a document-term matrix, $\Theta$, whose rows sum to 1 and represent the distribution over topics of a document, and a term-topic matrix $\mathbf{\beta}$, whose columns sum to 1 and represent the distrubtion of words of a topic k.

Ultimately, we would like to find out for a word in a document, what is the probability that the word belongs to a topic k, $P(z_i=j|\mathbf{z}_{-i})$, where $z_i$ represents the assignment of word $i$ and $z_{-i}$ is the assignment of all other words.

This is where Gibbs sampling will help us. 

* Step 1-We'll start by randomly assigning all the words in every document to a topic, 1,2,...K. 
* Step 2-Compute two matrices: 
    * Document-Topic Counts: Each row is a document. Each columns is a topic. Element i,j is the number of words in document i assigned to topic j.
    * Term-Topic Counts: Each row is a term in our vocabular. Each column is a topic. Element i,j is the number of times in the entire corpus (across all documents) term i is assigned to topic j.
* Step 3-For each word in each document, compute for each k topic:
    * $P(z_i=j|\mathbf{z}_{-i})= \frac{n_{-i,j}^{w_i}+\beta}{n_{-i,j}^{corpus}+V\beta}* \frac{n_{-i,j}^{doc}+\alpha}{n_{-i}^{doc}+K\alpha}$, where
        * $n_{-i,j}^{w_i}$: number of times word $w_i$ was assigned to topic j, excluding current word
        * $n_{-i,j}^{corpus}$: number of words assigned to topic j across entire corpus
        * $n_{-i,j}^{doc}$: number of words in document assigned to topic j, excluding current word
        * $n_{-i}^{doc}$: number of words, excluding current word, in document
        * $\beta$ is a smoothing parameter, typically equal to 200/V
        * $\alpha$ is a smoothing paramets, typically 50/K.
    * We will have K values, for this specific word in this document, of $P(z_i=j|\mathbf{z}_{-i})$. We will normalize these probabilities so that they sum to 1, and then pick 1 out of the K topics at random. 
    * <b>Note</b> $\frac{n_{-i,j}^{w_i}+\beta}{n_{-i,j}^{corpus}+V\beta}$, is how much a topic likes a particular word. So if over the corpus this word shows up in that topic quite often, it will increase the probability topic K is chosen when we pick 1 topic at random.
    * <b>Note</b> $\frac{n_{-i,j}^{doc}+\alpha}{n_{-i}^{doc}+K\alpha}$, is how much a document likes a particular topic. So if a document uses a particular topic k in all other words in that document, it will increase the probability topic k is chosen when we pick 1 topic at random.
* Step 4- Repeat step 3 N times (this is where Gibbs sampling says we will approximate the joint distribution $P(\mathbf{z})$, the topic assignment of all words in all documents. 

## Code ##

Let's start off by defining the number of topics, number of unique words.

In [5]:
text = pd.Series(data)
tokens = list(set(chain(*text)))
V = len(tokens)
K = 3
alpha = 50.0/K
beta = 200.0/V

Now we'll create the Document-Topic Count matrix and the Term-Topic Count matrix, using pandas dataframes to make it easy to see the columns/indices.

As the first step suggests, we will create the Document-Topic Count matrix by initializing a matrix with random numbers, 1 through K, for each word in each document.

In [11]:
doc_topic = text.apply(lambda x: np.random.randint(0,K,size=(len(x),)))
doc_topic_counts = doc_topic.apply(lambda x: Counter(x)).apply(pd.Series)
doc_topic_counts = doc_topic_counts.fillna(0)

term_topic_count = pd.DataFrame(index=tokens,columns=range(K),
									data=np.zeros((V,K)))
for doc_ind in range(text.shape[0]):
	for (topic,word) in zip(doc_topic.ix[doc_ind],text.ix[doc_ind]):
		term_topic_count.loc[word,topic]+=1

print doc_topic.head()
print doc_topic_counts.head()
print term_topic_count.head()

0    [0, 2, 2, 1, 0]
1       [1, 0, 0, 2]
2    [2, 0, 2, 1, 2]
3       [1, 0, 0, 2]
4       [2, 1, 1, 2]
dtype: object
   0  1  2
0  2  1  2
1  2  1  1
2  1  1  3
3  2  1  1
4  0  2  2
             0  1  2
economics    2  0  0
playoffs     0  1  0
apple        0  0  2
AAPL         0  1  1
competition  1  0  1


Now we will iterate through each word in each document. For each step, we will remove the word from  <i>doc_topic_counts</i>, and then from the  <i>term_topic_count</i>. 

Then, we will compute $P(z_i=j|\mathbf{z}_{-i})= \frac{n_{-i,j}^{w_i}+\beta}{n_{-i,j}^{corpus}+V\beta}* \frac{n_{-i,j}^{doc}+\alpha}{n_{-i}^{doc}+K\alpha}$. 

Lastly, we will draw a topic using the multinomial $(P(z_i=Topic1|\mathbf{z}_{-i}),P(z_i=Topic2|\mathbf{z}_{-i}),...,P(z_i=TopicK|\mathbf{z}_{-i}))$, and then assign that word in that document to the topic. We will update both the  <i>doc_topic_counts</i> and <i>term_topic_count</i> matrices.

In [12]:
	for doc_ind,doc in enumerate(text):
		topics = doc_topic.ix[doc_ind]

		for word_ind,word in enumerate(doc):
			# Remove current word from current calculations
			doc_topic_counts.loc[doc_ind,topics[word_ind]]-=1
			term_topic_count.loc[word,topics[word_ind]]-=1

			# Find conditional probability 
			# Multiply how much a word likes a given topic by
			# 	how much a document likes that topic
			word_given_topic = (term_topic_count.ix[word]+beta)/\
								(doc_topic_counts.sum()+V*beta)
			topic_given_doc = (doc_topic_counts.ix[doc_ind]+alpha)/\
								(doc_topic_counts.sum(1).ix[doc_ind]+K*alpha)
			weights = word_given_topic*topic_given_doc
			weights = weights/weights.sum()

			new_topic = np.where(np.random.multinomial(1,weights)==1)[0][0]
			topics[word_ind] = new_topic

			# Add back the removed word to appropriate topic
			doc_topic_counts.loc[doc_ind,new_topic]+=1
			term_topic_count.loc[word,new_topic]+=1

		doc_topic.ix[doc_ind] = topics
        
print doc_topic.head()
print doc_topic_counts.head()
print term_topic_count.head()

0    [2, 0, 1, 0, 1]
1       [0, 0, 2, 0]
2    [2, 2, 1, 2, 0]
3       [2, 1, 0, 0]
4       [2, 0, 0, 2]
dtype: object
   0  1  2
0  2  2  1
1  3  0  1
2  1  1  3
3  2  1  1
4  2  0  2
             0  1  2
economics    2  0  0
playoffs     1  0  0
apple        0  2  0
AAPL         0  0  2
competition  0  1  1


You can see how the topic assignments within a particular document changed. 

We will implement hundreds or even thousands of iterations, each time getting a better approximation of the distribution. We do this finally in the class below.

In [15]:
#################################
### Author: Paul Soto 		  ###
### 		paul.soto@upf.edu ###
#								#
# This file is a class to #######
# run  Gibbs                #
# sampling for Latent Dirichlet #
# Dirichlet Allocation  		#
#################################

import pandas as pd
from Text_Preprocessing import *
import re
from itertools import chain
import numpy as np 
from collections import Counter

class Gibbs():
	"""
	A class for the uncollapsed Gibbs sampling on text
	"""
	def __init__(self, text, K=2):
		"""
		text: Pandas series with each row a list of words in the document
		K: number of topics
		"""
		self.tokens = list(set(chain(*text.values)))
		self.V = len(self.tokens)
		self.K = K
		self.text = text

		### Create objects we'll need in updating parameters
		self.doc_topic = text.apply(lambda x: np.random.randint(0,self.K,size=(len(x),)))
		self.doc_topic_counts = self.doc_topic.apply(lambda x: Counter(x)).apply(pd.Series)
		self.doc_topic_counts = self.doc_topic_counts.fillna(0)

		# Fill missing columns (typically if K is too large)
		if list(self.doc_topic_counts.columns)!=range(self.K):
			needed = [el for el in range(self.K) if el not in self.doc_topic_counts.columns]
			for col in needed:
				self.doc_topic_counts[col] = 0

		self.term_topic_count = pd.DataFrame(index=self.tokens,columns=range(self.K),
											data=np.zeros((self.V,self.K)))
		for doc_ind in range(self.text.shape[0]):
			for (topic,word) in zip(self.doc_topic.ix[doc_ind],self.text.ix[doc_ind]):
				self.term_topic_count.loc[word,topic]+=1

		# Set priors
		self.alpha = 50.0/self.K
		self.beta = 200.0/self.V
		self.perplexity_scores = []

	def iterate(self,n=1000):
		"""
		Run n steps of the Gibbs sampler

		Relies on two calculations: 
			word_given_topic: "probability" of observing a word given a topic
			topic_given_doc: "probability" of observing topic j 

			Each is calculated by removing the current word from document
		"""
		for step in range(n):
			if step%25==0: 
				print "Step %s of Gibbs Sampling Completed" % step
				self.perplexity()

			for doc_ind,doc in enumerate(self.text):
				topics = self.doc_topic.ix[doc_ind]

				for word_ind,word in enumerate(doc):
					# Remove current word from current calculations
					self.doc_topic_counts.loc[doc_ind,topics[word_ind]]-=1
					self.term_topic_count.loc[word,topics[word_ind]]-=1

					# Find conditional probability 
					# Multiply how much a word likes a given topic by
					# 	how much a document likes that topic
					word_given_topic = (self.term_topic_count.ix[word]+self.beta)/\
										(self.doc_topic_counts.sum()+self.V*self.beta)
					topic_given_doc = (self.doc_topic_counts.ix[doc_ind]+self.alpha)/\
										(self.doc_topic_counts.sum(1).ix[doc_ind]+self.K*self.alpha)
					weights = word_given_topic*topic_given_doc
					weights = weights/weights.sum()

					new_topic = np.where(np.random.multinomial(1,weights)==1)[0][0]
					topics[word_ind] = new_topic

					# Add back the removed word to appropriate topic
					self.doc_topic_counts.loc[doc_ind,new_topic]+=1
					self.term_topic_count.loc[word,new_topic]+=1

				self.doc_topic.ix[doc_ind] = topics

	def perplexity(self):
		"""
		Compute perplexity scores of samples (currently insample)
		"""
		dt = (self.doc_topic_counts+self.alpha).apply(lambda x: x/x.sum(),1).fillna(0)
		tt = (self.term_topic_count+self.beta)/(self.term_topic_count+self.beta).sum().fillna(0)

		def prob(row):
			word_list = row[0]
			index = row['index']
			doc_perp= 0
			for each in word_list:
				doc_perp+=np.log((tt.ix[each]*dt.ix[index]).sum())
			return doc_perp

		perplexity = self.text.reset_index().apply(prob,1)
		perplexity = perplexity.sum()
		self.perplexity_scores.append(np.exp( - np.sum(perplexity) / self.text.apply(len).sum()))

	def top_n_words(self,n=10):
		"""
		Returns the n most frequent words from each topic
		"""

		for topic in range(self.K):
			top_n = self.term_topic_count.sort(topic,ascending=False)[topic].head(n)
			print "Top %s terms for topic %s" % (n,topic)

			for word in top_n.index: print word

In [19]:
data = [["rugby","football","competition","ball","games"],
        ["macro","economics","competition","games"],
        ["technology","computers","apple","AAPL","internet"],
        ["football","score","touchdown","team"],
        ["keynes","macro","friedman","policy"],
        ["stocks","AAPL","gains","analysis"],
        ["playoffs","games","season","compete","ball"],
        ["analysis","economy","economics","government"],
        ["apple","team","jobs","compete","computers"]]


gibbsobj = Gibbs(pd.Series(data),K=3)
gibbsobj.iterate(n=500)

Step 0 of Gibbs Sampling Completed
Step 25 of Gibbs Sampling Completed
Step 50 of Gibbs Sampling Completed
Step 75 of Gibbs Sampling Completed
Step 100 of Gibbs Sampling Completed
Step 125 of Gibbs Sampling Completed
Step 150 of Gibbs Sampling Completed
Step 175 of Gibbs Sampling Completed
Step 200 of Gibbs Sampling Completed
Step 225 of Gibbs Sampling Completed
Step 250 of Gibbs Sampling Completed
Step 275 of Gibbs Sampling Completed
Step 300 of Gibbs Sampling Completed
Step 325 of Gibbs Sampling Completed
Step 350 of Gibbs Sampling Completed
Step 375 of Gibbs Sampling Completed
Step 400 of Gibbs Sampling Completed
Step 425 of Gibbs Sampling Completed
Step 450 of Gibbs Sampling Completed
Step 475 of Gibbs Sampling Completed


In [21]:
print gibbsobj.top_n_words(5)

Top 5 terms for topic 0
games
touchdown
technology
policy
playoffs
Top 5 terms for topic 1
ball
team
analysis
computers
rugby
Top 5 terms for topic 2
economics
games
macro
apple
AAPL
None


We see the topics aren't necessarily coherent, but with more iterations they should converge to easily understood topics. 